✅ How to Use

    Start your gRPC display server (python server.py).

    Run the Jupyter notebook.

    Upload an image and click "Send to Display Server".

    The image and label will appear in the Gradio UI.

# 📦 Step 1: Import required packages

In [ ]:

import grpc
import display_pb2
import display_pb2_grpc
import cv2
import numpy as np
import os
# For image upload and preview
from PIL import Image
import io
from IPython.display import display
import ipywidgets as widgets


# 📸 Step 2: Define helper functions

In [ ]:

def encode_image_to_bytes(image_path):
    """Read an image and encode it into bytes (JPEG)"""
    img = cv2.imread(image_path)
    _, buffer = cv2.imencode('.jpg', img)
    return buffer.tobytes()

def preview_image(image_bytes):
    """Display an image in the notebook from bytes"""
    image = Image.open(io.BytesIO(image_bytes))
    display(image)

def encode_image(image_path: str) -> bytes:
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image file not found: {image_path}")
    img = cv2.imread(image_path)
    success, buffer = cv2.imencode('.jpg', img)
    if not success:
        raise ValueError("Failed to encode image")
    return buffer.tobytes()

# 📤 Step 3: Send image + label to gRPC display server

In [ ]:



def get_image_to_server(host='localhost', port=8061):
    channel = grpc.insecure_channel(f"{host}:{port}")
    stub = display_pb2_grpc.DisplayServiceStub(channel)

    request = display_pb2.AcquireRequest()

    response = stub.acquire(request)
    print("✅ Image required from  server.")
    return response

def send_image_to_server(image_bytes, label, host='localhost', port=8061):
    channel = grpc.insecure_channel(f"{host}:{port}")
    stub = display_pb2_grpc.DisplayServiceStub(channel)

    request = display_pb2.DisplayRequest(
        label=label,
        image=image_bytes
    )

    response = stub.display(request)
    print("✅ Image sent to display server.")
    return response


# 🖼️ Step 4: Upload widget to test with your own image

In [ ]:


uploader = widgets.FileUpload(
    accept='image/*',
    multiple=False
)

label_input = widgets.Text(
    value='Test image',
    description='Label:',
    disabled=False
)

button = widgets.Button(description="Send to Display Server")

def on_click(b):
    if uploader.value:
        file_info = next(iter(uploader.value.values()))
        image_bytes = file_info['content']
        label = label_input.value
        preview_image(image_bytes)
        send_image_to_server(image_bytes, label)
    else:
        print("⚠️ Please upload an image first.")

button.on_click(on_click)

display(widgets.VBox([uploader, label_input, button]))


In [ ]:
for a in range(15):
    r=get_image_to_server(host="printart.isr.ist.utl.pt")
    ro=send_image_to_server(r.image,r.label,host="printart.isr.ist.utl.pt")
    print(a)
    

In [ ]:

image_bytes = encode_image("/Users/jpc/Downloads/mathilde.jpg")
r=send_image_to_server(image_bytes,"merda")

In [ ]:
ro=send_image_to_server(r.image,r.label)

## Multiple images and Tabs


In [ ]:
import grpc
import json
import display_pb2
import display_pb2_grpc

In [ ]:


def load_image(path: str) -> bytes:
    """Load image from disk as raw bytes."""
    with open(path, "rb") as f:
        return f.read()


def save_image(image_bytes: bytes, path: str):
    """Save raw bytes to file."""
    with open(path, "wb") as f:
        f.write(image_bytes)


def run(server_address="localhost:50051"):
    # Choose 3 images (replace with your paths)
    image_paths = ["img1.jpg", "img2.jpg", "img3.jpg"]
    images = [load_image(p) for p in image_paths]

    # Create channel + stub
    channel = grpc.insecure_channel(server_address)
    stub = display_pb2_grpc.DisplayServiceStub(channel)

    # Build request
    request = display_pb2.DisplayRequest(
        label=json.dumps({"test": "multi-image"}),
        image=images
    )

    # Call service
    response = stub.process(request)

    # Save returned annotated images
    for i, img_bytes in enumerate(response.image):
        save_image(img_bytes, f"annotated_{i+1}.jpg")

    # Parse detections JSON
    detections = json.loads(response.config_json)  # depending on server return
    # if using YOLOResponse, it would be response.detections_json instead

    print("\nDetections per image:")
    for i, det_list in enumerate(detections):
        print(f"Image {i+1}:")
        for det in det_list:
            print(f"  {det}")

    print("\nAnnotated images saved as annotated_1.jpg, annotated_2.jpg, annotated_3.jpg")


if __name__ == "__main__":
    run()
